## Introduction to Text Mining

In this lab we look at a range of text mining techniques, available as part of Scikit-learn.

As our sample corpus of text, we will read a collection of news articles. These articles have been stored in a single file and formatted so that one article appears on each line.

In [ ]:
fin = open("news-articles.txt","r")
raw_documents = fin.readlines()
fin.close()
print("Read %d raw text documents" % len(raw_documents))

### Tokenizing Text

Raw text documents are textual, not numeric. The first step in analysing unstructured documents is to split the raw text into individual tokens, each corresponding to a single term (word). As an example:

In [ ]:
doc1 = raw_documents[0]
# print a snippet
print(doc1[0:300])

We will use the built-in scikit-learn tokenizer to split this document into tokens. Note that we will perform *case conversion* first to convert the entire text to lowercase.

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
tokenize = CountVectorizer().build_tokenizer()
# convert to lowercase, then tokenize
tokens1 = tokenize(doc1.lower())
print(tokens1)

We immediately see that many of the words here are not useful (e.g. "to", "the" etc.). Scikit-learn provides a list of such *stop words*:

In [ ]:
from sklearn.feature_extraction import text
stopwords = text.ENGLISH_STOP_WORDS
print(stopwords)

We can filter out these stopwords from our document:

In [ ]:
filtered_tokens1 = []
for token in tokens1:
    if not token in stopwords:
        filtered_tokens1.append(token)
print(filtered_tokens1)

We will repeat this process for all documents:

In [ ]:
all_filtered_tokens = []
for doc in raw_documents:
    # tokenize the next document
    tokens = tokenize(doc.lower())
    # remove the stopwords
    filtered_tokens = []
    for token in tokens:
        if not token in stopwords:
            filtered_tokens.append(token)  
    # add to the overall list
    all_filtered_tokens.append( filtered_tokens )
print("Created %d filtered token lists" % len(all_filtered_tokens) )

### Counting Tokens

A simple type of analysis that we might do is to count the number of times specific terms (words) appear in our corpus. We could do this by creating a dictionary of term frequency counts:

In [ ]:
counts = {}
# process filtered tokens for each document
for doc_tokens in all_filtered_tokens:
    for token in doc_tokens:
        # increment existing?
        if token in counts:
            counts[token] += 1
        # a new term?
        else:
            counts[token] = 1
print("Found %d unique terms in this corpus" % len(counts))

We would like to find the terms in the dictionary with the highest counts. Python provides a convenient way of doing this: 

In [ ]:
import operator
sorted_counts = sorted(counts.items(), key=operator.itemgetter(1), reverse=True)

The above creates a list of tuple pairs, where the first value is the key (i.e. the term) and the second value is the value (i.e the count). Let's display the top 20 terms.

In [ ]:
for i in range(20):
    term = sorted_counts[i][0]
    count = sorted_counts[i][1]
    print( "%s (count=%d)" % ( term, count )  )

### Bag-of-Words Representation

In the *bag-of-words model*, each document is represented by a vector in a *m*-dimensional coordinate space, where *m* is number of unique terms across all documents. This set of terms is called the corpus *vocabulary*. Note that the positioning (context) of terms within the original document is lost in this model.

Since each document can be represented as a term vector, we can stack these vectors to create a full *document-term matrix*. We can easily create this matrix from a list of document strings using Scikit-learn:

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
vectorizer = CountVectorizer()
X = vectorizer.fit_transform(raw_documents)
print(X.shape)

This process also build a vocabulary for the corpus, both in the form of a list and in the form of a dictionary:

In [ ]:
terms = vectorizer.get_feature_names()
vocab = vectorizer.vocabulary_
print("Vocabulary has %d distinct terms" % len(terms))

Display some sample terms:

In [ ]:
print(terms[500:530])

Since each column in the document-term matrix correspond to a term, we can look up the column associated with each term using the dictionary:

In [ ]:
# what column is the term 'football' on?
vocab["football"]

In [ ]:
# what column is the term 'world' on?
vocab["world"]

We can use the same Scikit-learn functionality to create a document-term matrix with N-grams. We specify an extra parameter ngram_range which specifies the shortest and longest token sequences to include. Length 1 is just a single token.

For instance, transform our input documents into a matrix, extracting single tokens and bigrams:

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
vectorizer = CountVectorizer(ngram_range = (1,2))
X = vectorizer.fit_transform(raw_documents)

Note the vocabulary is much larger now:

In [ ]:
terms = vectorizer.get_feature_names()
vocab = vectorizer.vocabulary_
print("Vocabulary has %d distinct terms" % len(terms))

Display some sample terms. Note that we see a mix of single tokens and bigrams (i.e. phrases of length 2):

In [ ]:
print(terms[510:520])

### Text Preprocessing

A range of steps can be used to process text input files to reduce the number of terms used to represent the text and to improve the resulting bag-of-words model. These include:
- Minimum term length: Exclude terms of length < 2. Scikit-learn does this by default.
- Case conversion: Converting all terms to lowercase. Scikit-learn does this by default.
- Stop-word filtering: Remove terms that appear on a pre-defined "blacklist" of terms that are highly frequent and do- not convey useful information.
- Low frequency filtering: Remove terms that appear in very few documents.
- Stemming: Reduce words to their stems (or base forms).

Scikit-learn allows us to perform one or more of these steps by adapting the CountVectorizer.

We can use the built-in list of stop-words for a given language by just specifying the name of the language (lower-case):

In [ ]:
vectorizer = CountVectorizer(stop_words="english")
X = vectorizer.fit_transform(raw_documents)
# Are standard stopwords gone?
"and" in vectorizer.vocabulary_

Or we could use our own custom stop-word list, which might be more appropriate for specific applications:

In [ ]:
custom_stop_words = [ "and", "the", "game" ] 
vectorizer = CountVectorizer(stop_words=custom_stop_words)
X = vectorizer.fit_transform(raw_documents)
# Are custom stopwords gone?
"game" in vectorizer.vocabulary_

We can remove low frequency terms that appear in fewer than a specified number of documents:

In [ ]:
# how many terms did we have with the last approach?
vectorizer = CountVectorizer()
X = vectorizer.fit_transform(raw_documents)
print("Number of terms in model is %d" % len(vectorizer.vocabulary_) )

In [ ]:
# build another matrix, but filter terms appearing in less than 5 documents
vectorizer = CountVectorizer(min_df = 5)
X = vectorizer.fit_transform(raw_documents)
print("Number of terms in model is %d" % len(vectorizer.vocabulary_) )

To stem tokens to their base form, we need to use functionality from another third party library: **NLTK**. You may need to install this package manually, either through the Conda interface or via the Conda command line tool, using:
    
    conda install nltk

We can test out the standard English stemming algorithm (called the Porter Stemmer):

In [ ]:
# import the standard English stemming algorithm
from nltk.stem.porter import PorterStemmer
words = ['plotted', 'flies', 'denied', 'sales', 'seas', 'computing', 'computed']
# try stemming each sample word
stemmer = PorterStemmer()
for w in words:
    print( stemmer.stem(w) )

To use NLTK stemming with Scikit-learn, we need to create a custom tokenisation function:

In [ ]:
import nltk
# define the function
def stem_tokenizer(text):
    # use the standard scikit-learn tokenizer first
    standard_tokenizer = CountVectorizer().build_tokenizer()
    tokens = standard_tokenizer(text)
    # then use NLTK to perform stemming on each token
    stemmer = PorterStemmer()
    stems = []
    for token in tokens:
        stems.append( stemmer.stem(token) )
    return stems

Now we can use our custom tokenizer with the standard CountVectorizer approach:

In [ ]:
vectorizer = CountVectorizer(tokenizer=stem_tokenizer)
X = vectorizer.fit_transform(raw_documents)
# display some sample terms
terms = vectorizer.get_feature_names()
print(terms[200:220])

We can perform lemmatisation in the same way, using NLTK with Sckit-learn:

In [ ]:
# define the function
def lemma_tokenizer(text):
    # use the standard scikit-learn tokenizer first
    standard_tokenizer = CountVectorizer().build_tokenizer()
    tokens = standard_tokenizer(text)
    # then use NLTK to perform lemmatisation on each token
    lemmatizer = nltk.stem.WordNetLemmatizer()
    lemma_tokens = []
    for token in tokens:
        lemma_tokens.append( lemmatizer.lemmatize(token) )
    return lemma_tokens

Again we can use our custom tokenizer with the standard CountVectorizer approach. The output terms are somewhat easier to intrepret than those produced by stemming:

In [ ]:
vectorizer = CountVectorizer(tokenizer=lemma_tokenizer)
X = vectorizer.fit_transform(raw_documents)
# display some sample terms
print(list(vectorizer.vocabulary_.keys())[0:35])

Let's put all of these steps together:

In [ ]:
vectorizer = CountVectorizer(stop_words="english",min_df = 3,tokenizer=lemma_tokenizer)
X = vectorizer.fit_transform(raw_documents)
print(X.shape)

In [ ]:
print(list(vectorizer.vocabulary_.keys())[0:35])

### Term Weighting

As well as including/excluding terms, we can also modify or weight the frequency values themselves. We can improve the usefulness of the document-term matrix by giving more weight to the more "important" terms.

The most common normalisation is *term frequency–inverse document frequency* (TF-IDF). In Scikit-learn, we can generate at TF-IDF weighted document-term matrix by using TfidfVectorizer() in place of CountVectorizer().

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
# we can pass in the same preprocessing parameters
vectorizer = TfidfVectorizer(stop_words="english",min_df = 5)
X = vectorizer.fit_transform(raw_documents)
# display some sample weighted values
print(X[0])

### Measuring Similarity

*Cosine similarity*: Most common approach for measuring similarity between two documents in a bag-of-words representation is to look at the cosine of the angle between their corresponding two term vectors. The motivation is that vectors for documents containing similar terms will point in the same direction in the m-dimensional vector space.

As an example, let's find the most similar document to the first document in our collection.

In [ ]:
# First document - just display the start of it
print(raw_documents[0][0:300])

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity
# Measure the cosine similarity between the first document vector and all of the others
max_cos = 0
best_row = 0
for row in range(1,X.shape[0]):
    cos = cosine_similarity( X[0], X[row] )
    # best so far?
    if cos > max_cos:
        max_cos = cos
        best_row = row

In [ ]:
print("Most similar document was row %d: cosine similarity = %.3f" % ( best_row, max_cos ) )

In [ ]:
# Best document - just display the start of it
print(raw_documents[best_row][0:300])